<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/Stock_NeurIPS2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Task Description](#0)
* [2. Install Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. A List of Python Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download and Preprocess Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5. Build Market Environment in OpenAI Gym-style](#4)  
    * [5.1. Data Split](#4.1)  
    * [5.3. Environment for Training](#4.2)    
* [6. Train DRL Agents](#5)
* [7. Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
  

<a id='0'></a>
# Part 1. Task Discription

We train a DRL agent for stock trading. This task is modeled as a Markov Decision Process (MDP), and the objective function is maximizing (expected) cumulative return.

We specify the state-action-reward as follows:

* **State s**: The state space represents an agent's perception of the market environment. Just like a human trader analyzing various information, here our agent passively observes many features and learns by interacting with the market environment (usually by replaying historical data).

* **Action a**: The action space includes allowed actions that an agent can take at each state. For example, a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying. When an action operates multiple shares, a ∈{−k, ..., −1, 0, 1, ..., k}, e.g.. "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* **Reward function r(s, a, s′)**: Reward is an incentive for an agent to learn a better policy. For example, it can be the change of the portfolio value when taking a at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio values at state s′ and s, respectively


**Market environment**: 30 consituent stocks of Dow Jones Industrial Average (DJIA) index. Accessed at the starting date of the testing period.


The data for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Install Python Packages

<a id='1.1'></a>
## 2.1. Install packages


In [2]:
## install finrl library
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


<a id='1.2'></a>
## 2.2. A list of Python packages 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

import sys
sys.path.append("../finrl")

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from env_stocktrading_pair_trading import StockPairTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import itertools

c:\Users\Administrador\AppData\Local\Programs\Python\Python38\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


<a id='1.4'></a>
## 2.4. Create Folders

In [260]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])



<a id='2'></a>
# Part 3. Download Data
Yahoo Finance provides stock data, financial news, financial reports, etc. Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** in FinRL-Meta to fetch data via Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).



-----
class YahooDownloader:
    Retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()


In [5]:
# from config.py, TRAIN_START_DATE is a string
TRAIN_START_DATE
# from config.py, TRAIN_END_DATE is a string
TRAIN_END_DATE

'2020-07-31'

In [6]:
TRAIN_START_DATE = '2015-10-19'
TRAIN_END_DATE = '2020-09-11' # 70% of All data set size (2556 aprox.)
TRADE_START_DATE = '2020-09-11'
TRADE_END_DATE = '2022-10-18'


In [7]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['AON','MMC']).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3524, 8)


In [8]:
# print(config_tickers.DOW_30_TICKER)

In [9]:
df.shape

(3524, 8)

In [10]:
df.sort_values(['date','tic'],ignore_index=True).head()
# Structure : date (%Y-%m-%d), open, high, low, close, volume, tic, day

,date,open,high,low,close,volume,tic,day
0,2015-10-19,91.570000,91.720001,90.379997,84.118172,1028600,AON,0
1,2015-10-19,53.900002,53.970001,53.540001,47.394081,1462100,MMC,0
2,2015-10-20,90.099998,91.599998,90.099998,84.981361,1791900,AON,1
3,2015-10-20,53.580002,54.380001,53.570000,47.879669,1874100,MMC,1
4,2015-10-21,92.070000,92.370003,91.230003,84.758583,1243100,AON,2


# Part 4: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

In [267]:
INDICATORS = ['close_10_sma']
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=False,
                    use_turbulence=False,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators


In [268]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [269]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,close_10_sma
0,2015-10-19,AON,91.570000,91.720001,90.379997,84.118172,1028600.0,0.0,84.118172
1,2015-10-19,MMC,53.900002,53.970001,53.540001,47.394081,1462100.0,0.0,47.394081
2,2015-10-20,AON,90.099998,91.599998,90.099998,84.981361,1791900.0,1.0,84.549767
3,2015-10-20,MMC,53.580002,54.380001,53.570000,47.879669,1874100.0,1.0,47.636875
4,2015-10-21,AON,92.070000,92.370003,91.230003,84.758583,1243100.0,2.0,84.619372
5,2015-10-21,MMC,54.430000,54.580002,53.910000,47.650135,1316800.0,2.0,47.641295
6,2015-10-22,AON,91.820000,93.339996,91.800003,85.770317,1016400.0,3.0,84.907108
7,2015-10-22,MMC,54.230000,55.270000,54.230000,48.559513,1849900.0,3.0,47.870850
8,2015-10-23,AON,92.900002,93.169998,91.620003,85.844536,1651400.0,4.0,85.094594
9,2015-10-23,MMC,55.200001,55.470001,54.889999,48.824390,1582600.0,4.0,48.061558


<a id='4'></a>
# Part 5. Build A Market Environment in OpenAI Gym-style
The training process involves observing stock price change, taking an action and reward's calculation. By interacting with the market environment, the agent will eventually derive a trading strategy that may maximize (expected) rewards.

Our market environment, based on OpenAI Gym, simulates stock markets with historical market data.

## Data Split
We split the data into training set and testing set as follows:

Training data period: 2009-01-01 to 2020-07-01

Trading data period: 2020-07-01 to 2021-10-31


In [270]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

2408
42


In [271]:
train.tail()

,date,tic,open,high,low,close,volume,day,close_10_sma
1201,2020-07-28,MMC,114.489998,114.980003,113.730003,110.476173,1435800.0,1.0,109.886129
1202,2020-07-29,AON,204.970001,206.539993,203.660004,201.728485,1462800.0,2.0,201.670663
1202,2020-07-29,MMC,113.839996,115.580002,113.150002,111.801384,1354300.0,2.0,110.316589
1203,2020-07-30,AON,203.639999,207.429993,202.610001,203.326248,1442900.0,3.0,202.137245
1203,2020-07-30,MMC,114.220001,117.050003,113.209999,112.894447,1721200.0,3.0,110.832269


In [272]:
trade.head()

,date,tic,open,high,low,close,volume,day,close_10_sma
0,2021-11-01,AON,316.929993,318.839996,306.059998,304.535858,1335900.0,0.0,314.099091
0,2021-11-01,MMC,167.000000,167.419998,161.740005,160.517349,2038400.0,0.0,164.789873
1,2021-11-02,AON,307.839996,308.890015,302.049988,302.401947,1608000.0,1.0,313.667505
1,2021-11-02,MMC,163.279999,164.789993,162.899994,161.799210,2565900.0,1.0,164.733669
2,2021-11-03,AON,305.779999,305.779999,294.589996,292.685181,1882000.0,2.0,311.831195


In [273]:
INDICATORS

['close_10_sma']

In [274]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 2, State Space: 7


In [275]:
print('>> ',train.shape)
train.head()

>>  (2408, 9)


,date,tic,open,high,low,close,volume,day,close_10_sma
0,2015-10-19,AON,91.570000,91.720001,90.379997,84.118172,1028600.0,0.0,84.118172
0,2015-10-19,MMC,53.900002,53.970001,53.540001,47.394081,1462100.0,0.0,47.394081
1,2015-10-20,AON,90.099998,91.599998,90.099998,84.981361,1791900.0,1.0,84.549767
1,2015-10-20,MMC,53.580002,54.380001,53.570000,47.879669,1874100.0,1.0,47.636875
2,2015-10-21,AON,92.070000,92.370003,91.230003,84.758583,1243100.0,2.0,84.619372


In [276]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

action_space = 3 # [-1:sell, 0:hold, 1:buy] # ==== OP1 
env_kwargs = {
    "hmax": 0,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": action_space, # ==== OP1 
    # "action_space":stock_dimension, # ==== OP2
    "reward_scaling": 1e-4,
    "print_verbosity" : 1,
    "model_name" : "test_pair_trading"
}

e_train_gym = StockPairTradingEnv(df = train, **env_kwargs)


In [277]:
stock_dimension

2

## Environment for Training



In [278]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Part 6: Train DRL Agents
* The DRL algorithms are from **Stable Baselines 3**. Users are also encouraged to try **ElegantRL** and **Ray RLlib**.
* FinRL includes fine-tuned standard DRL algorithms, such as DQN, DDPG, Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [279]:
agent = DRLAgent(env = env_train)

if_using_a2c = False
if_using_ddpg = True
if_using_ppo = False
if_using_td3 = False
if_using_sac = True


### Agent Training: 5 algorithms (A2C, DDPG, PPO, TD3, SAC)


### Agent 1: A2C


In [280]:
# agent = DRLAgent(env = env_train)
# model_a2c = agent.get_model("a2c")

# if if_using_a2c:
#   # set up logger
#   tmp_path = RESULTS_DIR + '/a2c'
#   new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
#   # Set new logger
#   model_a2c.set_logger(new_logger_a2c)


In [281]:
# trained_a2c = agent.train_model(model=model_a2c, 
#                              tb_log_name='a2c',
#                              total_timesteps=50000) if if_using_a2c else None

In [282]:
env_train.observation_space

Box([-inf -inf -inf -inf -inf -inf -inf], [inf inf inf inf inf inf inf], (7,), float32)

### Agent 2: DDPG

In [283]:
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 10, "learning_rate": 0.001}

agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 10, 'learning_rate': 0.001}
Using cpu device
Logging to results/ddpg


In [285]:
trained_ddpg = agent.train_model(model=model_ddpg,
                                 tb_log_name='ddpg',
                                 total_timesteps=1240) if if_using_ddpg else None


self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[inde

c:\Git\FinRL_cq\tutorials\1-Introduction\env_stocktrading_pair_trading.py:283: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "index" which is no longer supported as of 3.3 and will become an error two minor releases later
  plt.savefig(


self.reward >>  0.018326923311606517 	 actions[index] ::  [-1]  :: 	amount ::  1000316.1302636947
self.reward >>  8.675232467940078e-05 	 actions[index] ::  [-1]  :: 	amount ::  1000448.9541925811
self.reward >>  -0.0005800444629741833 	 actions[index] ::  [-1]  :: 	amount ::  1000581.3258022422
self.reward >>  -0.0001393595081404783 	 actions[index] ::  [-1]  :: 	amount ::  1000715.6184216078
self.reward >>  0.00015884795798920095 	 actions[index] ::  [-1]  :: 	amount ::  1000850.2503277013
self.reward >>  -0.00043980398407438774 	 actions[index] ::  [-1]  :: 	amount ::  1000984.5563817937
self.reward >>  -0.0017793210834614003 	 actions[index] ::  [-1]  :: 	amount ::  1001119.5897624508
self.reward >>  -0.00031360992736881603 	 actions[index] ::  [-1]  :: 	amount ::  1001257.158527679
self.reward >>  0.0017069347057375127 	 actions[index] ::  [-1]  :: 	amount ::  1001395.1142654225
self.reward >>  -0.0018542420845129527 	 actions[index] ::  [-1]  :: 	amount ::  1001531.1700501437
sel

In [174]:
trained_ddpg.env.render()

[1000000, 198.4736785888672, 111.83040618896484, 0, 0]

In [38]:
type(model_ddpg)
# e_train_gym.rewards_memory

stable_baselines3.ddpg.ddpg.DDPG

In [130]:
agent.env.observation_space

Box([-inf -inf -inf -inf -inf], [inf inf inf inf inf], (5,), float32)

### Agent 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000) if if_using_td3 else None

### Agent 5: SAC

In [134]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 5, # 100000,
    "learning_rate": 0.001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 5, 'learning_rate': 0.001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device
Logging to results/sac


In [135]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=10) if if_using_sac else None

self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.reward >>  0.0 	 actions[index] ::  [0 0]  :: 	amount ::  1000000
self.r

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [131]:
e_trade_gym = StockPairTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [132]:
trade.head()

,date,tic,open,high,low,close,volume,day,vix,turbulence
0,2020-09-11,AON,202.259995,203.070007,199.500000,197.972702,1189200.0,4.0,26.870001,0.058884
0,2020-09-11,MMC,115.959999,116.449997,114.860001,111.994843,1023800.0,4.0,26.870001,0.058884
1,2020-09-14,AON,203.330002,204.419998,201.679993,198.385300,1053600.0,0.0,25.850000,1.259911
1,2020-09-14,MMC,116.839996,118.040001,116.470001,113.948814,1199300.0,0.0,25.850000,1.259911
2,2020-09-15,AON,202.279999,203.919998,201.679993,198.424576,1013400.0,1.0,25.590000,0.205182


In [133]:
trained_moedl = trained_sac
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_moedl, 
    environment = e_trade_gym)

self.reward >>  -0.00026765370635548606 	 actions[index] ::  [-1 -1]  :: 	amount ::  1000309.6575770112
self.reward >>  -0.0001977281623752788 	 actions[index] ::  [-1 -1]  :: 	amount ::  1000621.6793569719
self.reward >>  -0.00024843625717330727 	 actions[index] ::  [-1 -1]  :: 	amount ::  1000934.5327782136
self.reward >>  0.0006741010803263635 	 actions[index] ::  [-1 -1]  :: 	amount ::  1001248.109207718
self.reward >>  0.00039940944137051705 	 actions[index] ::  [-1 -1]  :: 	amount ::  1001559.9236756669
self.reward >>  0.0018423216461087577 	 actions[index] ::  [-1 -1]  :: 	amount ::  1001870.8777606583
self.reward >>  -0.0005603003776515834 	 actions[index] ::  [-1 -1]  :: 	amount ::  1002178.7125544281
self.reward >>  0.0009621032295166515 	 actions[index] ::  [-1 -1]  :: 	amount ::  1002487.3030004806
self.reward >>  0.005726640042872168 	 actions[index] ::  [-1 -1]  :: 	amount ::  1002794.6534463195
self.reward >>  -0.0009205603637732566 	 actions[index] ::  [-1 -1]  :: 	amou

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])